# Inspect Fraud Data

Purpose
- Have a look at what the teaching staff's has given us and come up with a plan to deal with this fraud problem

In [16]:
# import libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [17]:
# setup spark
spark = (
    SparkSession.builder.appName("aggregate data for first 3 final model variables")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

In [18]:
merchant_fraud = pd.read_csv('../data/tables/merchant_fraud_probability.csv')

In [19]:
data1 = spark.read.parquet("../data/tables/transactions_20210228_20210827_snapshot/")
data2 = spark.read.parquet("../data/tables/transactions_20210828_20220227_snapshot/")
data3 = spark.read.parquet("../data/tables/transactions_20220228_20220828_snapshot/")

In [20]:
merchant_info = spark.read.parquet("../data/tables/tbl_merchants.parquet")
merchant_info_df = merchant_info.toPandas()

In [21]:
merchant_info_df[merchant_info_df['merchant_abn'] == 19492220327]

,name,tags,merchant_abn
426,Commodo Ipsum Industries,"([jewelry, watch, Clock, and silverware shops]...",19492220327


In [22]:
data = data1.union(data2)
data = data.union(data3)

In [23]:
consumer_fraud = pd.read_csv('../data/tables/consumer_fraud_probability.csv', )

In [24]:
data[(data['user_id'] == 6228) & (data['order_datetime']=='2021-12-19')]

data[(data['merchant_abn'] == 19492220327) & (data['order_datetime']=='2021-11-28')]

user_id,merchant_abn,dollar_value,order_id,order_datetime
2030,19492220327,23892.49785359222,9ea350e2-f379-4a5...,2021-11-28
18928,19492220327,9504.129566927579,0f8a2b37-7121-4ff...,2021-11-28
21706,19492220327,1903.7191517251547,5bc645f5-73e0-487...,2021-11-28
8210,19492220327,3188.26336439501,3683ac77-529f-423...,2021-11-28
10146,19492220327,18392.695298418792,7bac2854-b0dc-4ec...,2021-11-28
12891,19492220327,18208.528408907903,baa6783c-80a6-408...,2021-11-28
14746,19492220327,16647.911240773756,987ca916-86ff-4b2...,2021-11-28


In [25]:
consumer_fraud

,user_id,order_datetime,fraud_probability
0,6228,2021-12-19,97.629808
1,21419,2021-12-10,99.247380
2,5606,2021-10-17,84.058250
3,3101,2021-04-17,91.421921
4,22239,2021-10-19,94.703425
...,...,...,...
34859,18466,2022-02-22,8.679873
34860,17552,2021-12-26,8.349463
34861,17875,2021-06-27,8.288847
34862,10401,2021-09-18,8.842216


In [26]:
consumer_fraud.describe()

,user_id,fraud_probability
count,34864.000000,34864.000000
mean,12057.003930,15.120091
std,6963.195641,9.946085
min,1.000000,8.287144
25%,6059.750000,9.634437
50%,12067.500000,11.735624
75%,18091.250000,16.216158
max,24081.000000,99.247380


In [27]:
consumer_fraud.nunique()

user_id              20128
order_datetime         365
fraud_probability    34765
dtype: int64

In [28]:
merchant_fraud

,merchant_abn,order_datetime,fraud_probability
0,19492220327,2021-11-28,44.403659
1,31334588839,2021-10-02,42.755301
2,19492220327,2021-12-22,38.867790
3,82999039227,2021-12-19,94.134700
4,90918180829,2021-09-02,43.325517
...,...,...,...
109,80089686333,2021-09-01,67.505811
110,83220249221,2021-11-27,63.512459
111,85482742429,2021-11-27,70.881311
112,11470993597,2021-09-28,63.377344


In [29]:
merchant_fraud.nunique()

merchant_abn          61
order_datetime        64
fraud_probability    113
dtype: int64

In [30]:
merchant_fraud.describe()

,merchant_abn,fraud_probability
count,1.140000e+02,114.000000
mean,5.440738e+10,40.419335
std,3.172677e+10,17.187745
min,1.114906e+10,18.210891
25%,2.173151e+10,28.992765
50%,4.942497e+10,32.692032
75%,8.491712e+10,48.395260
max,9.998904e+10,94.134700
